##### Fernando Amaral
##### FPGrowth

In [ ]:
# association rules

In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("fpgrowth").getOrCreate()

In [2]:
from pyspark.ml.fpm import FPGrowth

In [4]:
from pyspark.sql.functions import split
transacoes = spark.read.text("Transacoes.txt").select(split("value","\s+").alias("transacoes"))
transacoes.show(truncate=False)

+-------------------------+
|transacoes               |
+-------------------------+
|[Cerveja, Pizza, Sorvete]|
|[Pizza, Sorvete]         |
|[Cerveja, Pizza]         |
|[Cerveja, Pizza, Sorvete]|
|[Cerveja, Pizza]         |
|[Pizza]                  |
+-------------------------+



In [6]:
fp = FPGrowth(itemsCol="transacoes", minSupport=0.5, minConfidence=0.5)
modelo = fp.fit(transacoes)
modelo.freqItemsets.show()

+----------------+----+
|           items|freq|
+----------------+----+
|       [Sorvete]|   3|
|[Sorvete, Pizza]|   3|
|       [Cerveja]|   4|
|[Cerveja, Pizza]|   4|
|         [Pizza]|   6|
+----------------+----+



In [7]:
modelo.associationRules.show()

+----------+----------+------------------+----+------------------+
|antecedent|consequent|        confidence|lift|           support|
+----------+----------+------------------+----+------------------+
| [Cerveja]|   [Pizza]|               1.0| 1.0|0.6666666666666666|
| [Sorvete]|   [Pizza]|               1.0| 1.0|               0.5|
|   [Pizza]| [Sorvete]|               0.5| 1.0|               0.5|
|   [Pizza]| [Cerveja]|0.6666666666666666| 1.0|0.6666666666666666|
+----------+----------+------------------+----+------------------+



In [8]:
modelo.transform(transacoes).show(truncate=False)

+-------------------------+------------------+
|transacoes               |prediction        |
+-------------------------+------------------+
|[Cerveja, Pizza, Sorvete]|[]                |
|[Pizza, Sorvete]         |[Cerveja]         |
|[Cerveja, Pizza]         |[Sorvete]         |
|[Cerveja, Pizza, Sorvete]|[]                |
|[Cerveja, Pizza]         |[Sorvete]         |
|[Pizza]                  |[Sorvete, Cerveja]|
+-------------------------+------------------+

